In [1]:
##import
from binance.client import Client
import pandas as pd
import os
from finlab import crypto
import talib
from binance.enums import *
import math
import time
from tqdm import tqdm
import threading
##顯示pandas所有行與列
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

##api設置
api_key=os.environ.get('binance_api')
api_secret=os.environ.get('binance_secret')
client = Client(api_key, api_secret)

c:\users\cheng\appdata\local\programs\python\python39\lib\site-packages\swagger_spec_validator\validator20.py:48: SwaggerValidationWarning: Found "$ref: #/definitions/UserPreferences" with siblings that will be overwritten. See https://stackoverflow.com/a/48114924 for more information. (path #/definitions/User/properties/preferences)
  warnings.warn(
c:\users\cheng\appdata\local\programs\python\python39\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
### Function

## Set COHLV_data Function
def set_COHLV_data(Dataframe):
    COHLV={'Close':pd.Series(Dataframe['Close']),
            'High':pd.Series(Dataframe['High']),
            'Low':pd.Series(Dataframe['Low']),
            'Open':pd.Series(Dataframe['Open'])}
    return COHLV


## get_margin_assets_detail Function
def get_margin_assets_detail():
    margin_assets_info = client.get_margin_account() 
    margin_account_detail=pd.DataFrame()
    x=0
    for i in margin_assets_info['userAssets']:
        margin_account_detail=pd.concat([margin_account_detail,pd.DataFrame([i])],axis=0)
    margin_account_detail.set_index('asset', inplace = True)
    margin_account_detail.drop(margin_account_detail[(margin_account_detail.netAsset == '0')].index, inplace=True)
    return(margin_account_detail)

def set_loan_detail():
    loan_detail={'APE_max_loan_amount':client.get_max_margin_loan(asset='APE'),
                'BUSD_max_loan_amount':client.get_max_margin_loan(asset='BUSD'),
                'APE_current_loan':str(float(margin_account_detail['borrowed']['APE'])+float(margin_account_detail['interest']['APE'])) ,
                'BUSD_current_loan':str(float(margin_account_detail['borrowed']['BUSD'])+float(margin_account_detail['interest']['BUSD'])),
                }
    return(loan_detail)



## Strategy_function

def EMA_Strategy(COHLV_data,fast=14,slow=26):
    ##計算EMA快慢線
    EMA_fast=talib.EMA(COHLV_data['Close'],fast)
    EMA_slow=talib.EMA(COHLV_data['Close'],slow)
    ##判斷長部頭寸及短部頭寸
    signal_long = (EMA_fast > EMA_slow) & (EMA_fast.shift() < EMA_slow.shift())
    signal_short = (EMA_slow > EMA_fast) & (EMA_slow.shift() < EMA_fast.shift())
    signal = EMA_fast-EMA_slow
    return signal

def KD_Strategy(COHLV_data,ftk=10,sk=2,sd=4):
    KD=talib.STOCH(COHLV_data['High'],
                COHLV_data['Low'],
                COHLV_data['Close'],
                fastk_period=ftk,
                slowk_period=sk,
                slowd_period=sd)
    signal = KD[0]-KD[1]
    return signal
    
# UlTMATE PROJECT-Operation Arclight START Function
def set_interval(func, sec):
    def func_wrapper():
        set_interval(func, sec)
        func()
    t = threading.Timer(sec, func_wrapper)
    t.start()
    return t

In [3]:
def Operation_Arclight():
    ##取得APEBUSD最新資料(30分K)
    global margin_account_detail,loan_detail,COHLV
    print('----------------------------------------------------------------------------------------------------------------------------------')
    df = crypto.get_all_binance('APEBUSD', '30m')
    df = df.drop(['Close_time','Quote_av','Trades','Tb_base_av','Tb_quote_av','Ignore'],axis=1)
    ##設定COHLV變數
    COHLV=set_COHLV_data(df)

    ##print目前槓桿帳戶資料
    print('\n槓桿帳戶資產總覽(已隱藏為零資產)\n')
    margin_account_detail = get_margin_assets_detail()
    print(margin_account_detail)
    loan_detail = set_loan_detail()
    print('★ 您目前帳戶權益為 ${} BUSD ★'.format(str(math.floor(float(client.get_recent_trades(symbol='BTCBUSD')[0]['price'])*float(client.get_margin_account()['totalNetAssetOfBtc'])*100)/100)))
    print('=====您目前可貸款 {} APE，尚未償還 {} APE====='.format(loan_detail['APE_max_loan_amount']['amount'],loan_detail['APE_current_loan']))
    print('=====您目前可貸款 {} BUSD，尚未償還 {} BUSD====='.format(loan_detail['BUSD_max_loan_amount']['amount'],loan_detail['BUSD_current_loan']))

    try: ##設定策略
        
        signal = EMA_Strategy(COHLV_data=COHLV)
        
        print('\n執行Operation Arclight計畫')
        print('最近資料時間(UTC+0)：')
        print(signal.index[-1])
        ##判斷當下是否做多or保持不變or做空
        if signal[-1] > 0:
            print('▲ 保持多頭頭寸 ▲')
        elif signal[-1] < 0:
            print('▼ 保持空頭頭寸 ▼')
        ##繪製圖表
        #EMA=pd.concat([EMA_fast, EMA_slow], axis=1)
        #EMA.plot(figsize=(25,10))
        #signal.plot()
        if signal[-1] > 0:
            for i in range(2):
                print('建立多頭頭寸 開始')

                print('Step1.買進空頭頭寸還款額')
                try:

                    loan_detail = set_loan_detail()
                    margin_account_detail = get_margin_assets_detail()

                    order = client.create_margin_order(
                        symbol='APEBUSD',
                        side='BUY',
                        type='MARKET',
                        quantity=str(math.ceil(float(loan_detail['APE_current_loan']) * 100) / 100.0))
                    print('已買入 {} APE，成交額 {}，成交均價 {}，訂單編號 {}'.format(order['fills'][0]['qty'],order['cummulativeQuoteQty'],order['fills'][0]['price'],order['orderId']))
                    with tqdm(total=10) as pbar:
                        for i in range(10):
                            time.sleep(1)
                            pbar.update(1)

                except:
                    print('無須買進空頭頭寸還款額')


                print('Step2.償還空頭頭寸已欠款額')
                try:

                    loan_detail = set_loan_detail()
                    margin_account_detail = get_margin_assets_detail()

                    transaction = client.repay_margin_loan(asset='APE', amount=loan_detail['APE_current_loan'])
                    print('已償還所有 APE，流水編號{}'.format(transaction['tranId']))

                    with tqdm(total=10) as pbar:
                        for i in range(10):
                            time.sleep(1)
                            pbar.update(1)

                except:
                    print('錯誤 可能已償還空頭頭寸')

                print('Step3.借入最大多頭頭寸')
                try:

                    loan_detail = set_loan_detail()
                    margin_account_detail = get_margin_assets_detail()

                    transaction = client.create_margin_loan(asset='BUSD', amount=loan_detail['BUSD_max_loan_amount']['amount'])
                    print('已借入 BUSD 最大貸款金額，流水編號{}'.format(transaction['tranId']))

                    with tqdm(total=10) as pbar:
                        for i in range(10):
                            time.sleep(1)
                            pbar.update(1)

                except:
                    print('錯誤 目前貸款額可能已達上限')

                print('Step4.建立多頭頭寸')
                margin_account_detail = get_margin_assets_detail()
                try:
                    order = client.create_margin_order(
                        symbol='APEBUSD',
                        side='BUY',
                        type='MARKET',
                        quantity=str(math.ceil(math.floor(float(margin_account_detail['free']['BUSD'])/10) * 10 / COHLV['Close'][-1] * 100)/100)
                    ##買進BUSD帳戶無條件捨去至十位數可買APE總量
                    )
                    print('已買入 {} APE，成交額 {}，成交均價 {}，訂單編號 {}'.format(order['fills'][0]['qty'],order['cummulativeQuoteQty'],order['fills'][0]['price'],order['orderId']))
                    print('多頭頭寸 Operation Arclight 執行完畢')

                    loan_detail = set_loan_detail()
                    margin_account_detail = get_margin_assets_detail()

                except:
                    print('已無法買進更多 APE')

        if signal[-1] < 0:
                for i in range(2):
                    print('建立空頭頭寸 開始')

                    print('Step1.出售多頭頭寸')
                    try:

                        loan_detail = set_loan_detail()
                        margin_account_detail = get_margin_assets_detail()

                        order = client.create_margin_order(
                            symbol='APEBUSD',
                            side='SELL',
                            type='MARKET',
                            quantity=str(math.floor(float(margin_account_detail['free']['APE'])*100)/100))
                        print('已出售 {} APE，成交額 {}，成交均價 {}，訂單編號 {}'.format(order['fills'][0]['qty'],order['cummulativeQuoteQty'],order['fills'][0]['price'],order['orderId']))

                        with tqdm(total=10) as pbar:
                            for i in range(10):
                                time.sleep(1)
                                pbar.update(1)

                    except:
                        print('已無 APE 可供出售')

                    print('Step2.償還多頭頭寸已欠款額')
                    try:

                        loan_detail = set_loan_detail()
                        margin_account_detail = get_margin_assets_detail()

                        transaction = client.repay_margin_loan(asset='BUSD', amount=loan_detail['BUSD_current_loan'])
                        print('已償還所有 BUSD，流水編號{}'.format(transaction['tranId']))

                        with tqdm(total=10) as pbar:
                            for i in range(10):
                                time.sleep(1)
                                pbar.update(1)

                    except:
                        print('錯誤 可能已償還多頭頭寸')

                    print('Step3.借入空頭頭寸款項')
                    try:

                        loan_detail = set_loan_detail()
                        margin_account_detail = get_margin_assets_detail()

                        transaction = client.create_margin_loan(asset='APE', amount=loan_detail['APE_max_loan_amount']['amount'])
                        print('已借入APE最大貸款金額，流水編號{}'.format(transaction['tranId']))

                        with tqdm(total=10) as pbar:
                            for i in range(10):
                                time.sleep(1)
                                pbar.update(1)

                    except:
                        print('錯誤 目前貸款額可能已達上限')

                    print('Step4.建立空頭頭寸')
                    try:       
                        loan_detail = set_loan_detail()
                        margin_account_detail = get_margin_assets_detail()

                        order = client.create_margin_order(
                            symbol='APEBUSD',
                            side='SELL',
                            type='MARKET',
                            quantity=str(math.floor(float(margin_account_detail['free']['APE'])*100)/100))
                        print('已出售 {} APE，成交額 {}，成交均價 {}，訂單編號 {}'.format(order['fills'][0]['qty'],order['cummulativeQuoteQty'],order['fills'][0]['price'],order['orderId']))
                        print('空頭頭寸 Operation Arclight 執行完畢')

                    except:
                        print('已無法出售更多 APE')
    except:
        print('策略設定失敗')

# for i in range(10000):
#     Operation_Arclight()
#     print('距離下次判讀時間')
#     with tqdm(total=100) as pbar:
#         for i in range(100):
#             time.sleep(1)
#             pbar.update(1)
    

In [4]:
set_interval(Operation_Arclight,1800)

<Timer(Thread-7, started 12364)>

----------------------------------------------------------------------------------------------------------------------------------
2022-03-29 07:30:00 2022-03-29 16:09:35.112848
All caught up..!

槓桿帳戶資產總覽(已隱藏為零資產)

               free locked     borrowed    interest      netAsset
asset                                                            
BNB      0.00000526      0            0           0    0.00000526
USDT       0.304932      0            0           0      0.304932
APE      0.08774285      0  20.15306172  0.00100718  -20.06632605
BUSD   424.79058196      0            0           0  424.79058196
★ 您目前帳戶權益為 $140.53 BUSD ★
=====您目前可貸款 0 APE，尚未償還 20.1540689 APE=====
=====您目前可貸款 0 BUSD，尚未償還 0.0 BUSD=====

執行Operation Arclight計畫
最近資料時間(UTC+0)：
2022-03-29 08:00:00
▼ 保持空頭頭寸 ▼
建立空頭頭寸 開始
Step1.出售多頭頭寸
已無 APE 可供出售
Step2.償還多頭頭寸已欠款額
錯誤 可能已償還多頭頭寸
Step3.借入空頭頭寸款項
錯誤 目前貸款額可能已達上限
Step4.建立空頭頭寸
已無法出售更多 APE
建立空頭頭寸 開始
Step1.出售多頭頭寸
已無 APE 可供出售
Step2.償還多頭頭寸已欠款額
錯誤 可能已償還多頭頭寸
Step3.借入空頭頭寸款項
錯誤 目前貸款額可能已達

In [ ]:
##計算EMA快慢線
EMA_fast=talib.EMA(COHLV['Close'],9)
EMA_slow=talib.EMA(COHLV['Close'],26)
##判斷長部頭寸及短部頭寸
signal_long = (EMA_fast > EMA_slow) & (EMA_fast.shift() < EMA_slow.shift())
signal_short = (EMA_slow > EMA_fast) & (EMA_slow.shift() < EMA_fast.shift())
signal = signal_long
signal[signal_short] = -1
signal = signal.astype(int)
##判斷當下是否做多or保持不變or做空
print('\n執行Operation Arclight計畫')
print('最近資料時間(UTC+0)：')
print(signal.index[-1])
if signal[-1] == 1:
    print('▲ 多頭頭寸 ▲')
elif signal[-1] == 0:
    print('□ 保持倉位 □')
elif signal[-1] == -1:
    print('▼ 空頭頭寸 ▼')